#Grupo3 Laboratorio 7

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [2]:
b

array([0.76711663, 0.70811536, 0.79686718, 0.55776083, 0.96583653,
       0.1471569 , 0.029647  , 0.59389349, 0.1140657 , 0.95080985])

In [3]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [4]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [5]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [6]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [7]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [8]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [9]:
import numpy as np 

def jacobi_iteration(M_A, V_b, num_iterations):
  n = M_A.shape[0]
  x = np.zeros(n)
  norms_diff = []
  norms_B = []
  for _ in range(num_iterations):
      x_new = np.zeros(n)
      for i in range(n):
          x_new[i] = (V_b[i] - np.dot(M_A[i, :i], x[:i]) - np.dot(M_A[i, i+1:], x[i+1:])) / M_A[i, i]
      norms_diff.append(np.linalg.norm(x_new - x))
      norms_B.append(np.linalg.norm(M_A - np.diag(np.diagonal(M_A))))

      x = x_new

  return x, norms_diff, norms_B

In [10]:
A = np.array([[7, 1, 1],
              [1, 8, 5],
              [1, 5, 9]])

b = np.array([1, 5, 6])

num_iterations = 10  #iteraciones de la operacion

In [11]:
#extrayendo datos de la matriz 
x_orig, diff_orig, norm_B_orig = jacobi_iteration(A, b, num_iterations)

In [12]:
A_modified = np.array([[0.7, 0.1, 0.1],
                       [0.1, 0.8, 0.5],
                       [0.1, 0.5, 0.9]])
x_modified, diff_modified, norm_B_modified = jacobi_iteration(A_modified, b, num_iterations)


In [13]:
print('matriz original')
print(x_orig)
print(diff_orig)

print(norm_B_orig)

print('matriz modificada')
print(x_modified)
print(diff_modified)

print(norm_B_modified)

matriz original
[0.0259429  0.31264433 0.4827487 ]
[0.9249203250603538, 0.5955653862101966, 0.3795756829762065, 0.24489902652625958, 0.15635375893678347, 0.1007923899409298, 0.0644041091155266, 0.041489469087698225, 0.026529378746075756, 0.017080474264069775]
[7.3484692283495345, 7.3484692283495345, 7.3484692283495345, 7.3484692283495345, 7.3484692283495345, 7.3484692283495345, 7.3484692283495345, 7.3484692283495345, 7.3484692283495345, 7.3484692283495345]
matriz modificada
[0.25942903 3.12644329 4.82748699]
[9.249203250603538, 5.955653862101966, 3.7957568297620656, 2.448990265262597, 1.5635375893678343, 1.0079238994092972, 0.6440410911552651, 0.41489469087698144, 0.26529378746075644, 0.1708047426406971]
[0.7348469228349535, 0.7348469228349535, 0.7348469228349535, 0.7348469228349535, 0.7348469228349535, 0.7348469228349535, 0.7348469228349535, 0.7348469228349535, 0.7348469228349535, 0.7348469228349535]


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [14]:
#declarando funcion gauss_seidel_iteration
import numpy as np

def gauss_seidel_iteration(A, b, num_iterations):
    n = A.shape[0]
    x = np.zeros(n)  
    norms_diff = []
    norms_B = []
    for _ in range(num_iterations):
        x_new = np.zeros(n)
        for i in range(n):
            x_new[i] = (b[i] - np.dot(A[i, :i], x_new[:i]) - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
        norms_diff.append(np.linalg.norm(x_new - x))
        norms_B.append(np.linalg.norm(A - np.tril(A) @ np.triu(A, 1)))

        x = x_new

    return x, norms_diff, norms_B

In [15]:
#implamentando la funcion podemos aplicar el siguiente ejemplo
n = 7 
A = np.random.rand(n, n)  
b = np.random.rand(n)  
num_iterations = 20

In [16]:
x, diff, norm_B = gauss_seidel_iteration(A, b, num_iterations)
print(x)
print(diff)
print(norm_B)

[-4.60808837e+15 -2.57548897e+16  6.46731985e+16 -2.22300005e+16
 -9.48960122e+15  1.09740570e+17 -4.14557468e+16]
[4.200308349132486, 44.96420933430711, 364.92182404563704, 2711.664689262259, 19585.668772369507, 140072.3093500574, 998242.3567374678, 7105074.569337129, 50547790.92225247, 359553646.25572413, 2557402885.3921027, 18189684719.606625, 129374230055.61836, 920172286545.4698, 6544704958771.381, 46549051366625.914, 331078926319228.25, 2354790202648868.5, 1.6748383420674896e+16, 1.1912243608633808e+17]
[3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213, 3.6084255803323213]


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [17]:
# ... ENTER YOUR CODE HERE ...
#declarando la funcion minres_scheme
def minres_scheme(A, b, num_iterations):
    n = A.shape[0]
    x = np.zeros(n)  
    r = b - A @ x  
    p = r.copy()  
    norms_resid = []
    norms_deviation = []
    tau_values = []

    for k in range(num_iterations):
        Ap = A @ p
        denom = np.dot(p, Ap)
        if denom == 0:
            break
        tau = np.dot(r, Ap) / denom
        x_new = x + tau * p
        r_new = r - tau * Ap
        norms_resid.append(np.linalg.norm(r_new))
        norms_deviation.append(np.linalg.norm(x_new - x))
        tau_values.append(tau)
        x = x_new
        r = r_new
        beta = np.linalg.norm(r)
        p = r + (beta / np.linalg.norm(r)) * p

    return x, norms_resid, norms_deviation, tau_values

In [18]:
#aplicando el mismo ejemplo que Task II
#implamentando la funcion podemos aplicar el siguiente ejemplo
n = 7 
A = np.random.rand(n, n)  
b = np.random.rand(n)  
num_iterations = 20

In [19]:
x_ground_truth = np.linalg.solve(A, b)
print(x_ground_truth)
x_minres, norms_resid, norms_deviation, tau_values = minres_scheme(A, b, num_iterations)
print(x_minres)
print(norms_resid)
print(norms_deviation)
print(tau_values)

[ 1.63043583  1.75115962  0.23024805  0.8772116  -0.7806483  -1.46276723
 -1.70820815]
[-59428482.56773216 -73277902.7420925  -44030882.89636743
  41535384.1705648  -30685906.40937547  36304824.23075284
  16178421.48749717]
[2.5529529112060882, 8.536012696174675, 14.010818550701055, 983.3156460190081, 1708.862236654351, 4309.922158660675, 8974.404271913014, 17153.8025625905, 42783.263238991916, 69820.7982148689, 163213.72443860068, 331236.76769136154, 650753.6713897646, 1713512.873080285, 2635268.012653901, 6717945.228802531, 12450216.360741388, 28608450.374178737, 71068406.99113181, 107246999.10227348]
[1.4833104103634185, 6.134898317259387, 10.944228194956283, 734.9256265327731, 985.5591300180178, 2444.8619796721014, 4679.759308526854, 11992.09078788518, 23430.526365682374, 50285.700480880216, 95851.34263189188, 182756.94827417217, 464025.8791182982, 978170.0841532873, 2101579.7891546437, 4382407.97266955, 7480742.9621521635, 21008424.29773389, 41332086.724981725, 96510804.0668141]
[